In [1]:
using LinearAlgebra
using JuMP
using GLPK
using XLSX
using Complementarity #not sure I need this one?
using PATHSolver
using Revise #not having to restart sessions
#using Pandas #optional (only for EXIOBASE data)

include("../src/SUT_structure.jl") # Used for the SUT setup <sut = SUT.structure(...)>
include("../src/Constructs.jl") # Used to derive single-production systems from the SUT setup, e.g. as <itc = Constructs.ITC(sut)>
include("../src/Auxiliary.jl") # Includes some helper functions
include("../src/Model_data.jl") # Sets up the data structure for RCOT modelling based on SUT.structure or Constructs.construct
include("../src/RCOT_model.jl") # Builds and solves the RCOT models
include("../src/LP_models.jl"); # Builds and solves the LP models

In [2]:
# Load the workbook
xf = XLSX.readxlsx("../data/SUT_c-i+.xlsx")

XLSXFile("../data/SUT_c-i+.xlsx") containing 4 Worksheets
            sheetname size          range        
-------------------------------------------------
                    V 6x4           A1:D6        
                  U+e 4x7           A1:G4        
                    F 5x6           A1:F5        
                   pi 5x2           A1:B5        


In [3]:
# Get data from all worksheets and convert it to float64 data type
V = convert(Matrix{Float64}, xf["V!B2:D6"])
U = convert(Matrix{Float64}, xf["U+e!B2:F4"])
e = convert(Matrix{Float64}, xf["U+e!G2:G4"])
F = convert(Matrix{Float64}, xf["F!B2:F5"])
pii = convert(Matrix{Float64}, xf["pi!B2:B5"])
t = xf["V!A1"];

In [28]:
sut = SUT.structure(U,V,F,e,pii,t);

[ Info: A structure for supply-use elements was set up. Changing individual elements will not change others automatically.


In [8]:
itc = Constructs.ITC(sut)

construct([136.0; 96.0; 167.0;;], [51.0; 42.0; 31.0;;], [9.641472868217054 12.97168343454485 62.38684369723809; 31.21884735202492 15.000403984873474 7.780748663101605; 55.291530826631885 47.03340135357168 33.675067819796425], [0.07089318285453716 0.1351217024431755 0.37357391435471915; 0.22955034817665385 0.15625420817576535 0.04659130935988985; 0.4065553737252344 0.48993126409970494 0.20164711269339178], [1.59833317316061 0.7144586737028918 0.7896046410737875; 0.49661851260095824 1.4487520746555465 0.3169312490909092; 1.11870313292003 1.2528995188783316 1.8491729170383386], [11.41180057475428 4.148556760568855 11.939642664676864; 1.8050172265288542 2.3776251212691895 2.484024318868623; 2.3464303251306076 0.5423046132692892 5.259413209748251; 6.343125890506894 2.8536048675862875 12.231840670478245], [0.08391029834378147 0.043214132922592234 0.0714948662555501; 0.013272185489182752 0.024766928346554046 0.014874397118973792; 0.017253164155372116 0.005649006388221764 0.031493492273941624;

In [9]:
model_data = Model_data.IO(itc)

[ Info: You are setting up an IO model dataset. Elements of this dataset are now treated independently, meaning that no recalculation whatsoever takes place when individual elements are changed.


construct([136.0; 96.0; 167.0;;], [51.0; 42.0; 31.0;;], [9.641472868217054 12.97168343454485 62.38684369723809; 31.21884735202492 15.000403984873474 7.780748663101605; 55.291530826631885 47.03340135357168 33.675067819796425], [0.07089318285453716 0.1351217024431755 0.37357391435471915; 0.22955034817665385 0.15625420817576535 0.04659130935988985; 0.4065553737252344 0.48993126409970494 0.20164711269339178], [1.59833317316061 0.7144586737028918 0.7896046410737875; 0.49661851260095824 1.4487520746555465 0.3169312490909092; 1.11870313292003 1.2528995188783316 1.8491729170383386], [11.41180057475428 4.148556760568855 11.939642664676864; 1.8050172265288542 2.3776251212691895 2.484024318868623; 2.3464303251306076 0.5423046132692892 5.259413209748251; 6.343125890506894 2.8536048675862875 12.231840670478245], [0.08391029834378147 0.043214132922592234 0.0714948662555501; 0.013272185489182752 0.024766928346554046 0.014874397118973792; 0.017253164155372116 0.005649006388221764 0.031493492273941624;

In [49]:
model_data = Model_data.SU(sut);

[ Info: You are setting up an SU model dataset. Elements of this dataset are now treated independently, meaning that no recalculation whatsoever takes place when individual elements are changed.


In [5]:
# NST - MCP

model_data = sut
# To set up our initial primal, we write:
compl = Model(PATHSolver.Optimizer)

act = size(model_data.V,1)
com = size(model_data.V,2)
fac = size(model_data.F,1)

# Define the optimisation model
@variable(compl, z_star[1:act] >= 0)
@variable(compl, p_star[1:com] >= 0)
@variable(compl, pii_star[1:fac] >= 0)

@constraint(compl, c1, vec(pii_star'*model_data.F - (p_star'*(model_data.V' - model_data.U))) ⟂ z_star)
@constraint(compl, c2, vec((model_data.V' - model_data.U)*(z_star) - model_data.e) ⟂ p_star)
@constraint(compl, c3, vec((model_data.ϕ - model_data.F*z_star)) ⟂ pii_star)

# Solve
optimize!(compl)

Path 5.0.03 (Fri Jun 26 10:05:33 2020)
Written by Todd Munson, Steven Dirkse, Youngdae Kim, and Michael Ferris


Crash Log
major  func  diff  size  residual    step       prox   (label)
    0     0             1.4596e+02             0.0e+00 (f[    6])
    1    11     6     3 1.4595e+02  3.2e-04    1.5e+00 (f[    6])
    2    12     7     6 5.9007e+00  1.0e+00    1.3e+00 (f[   12])
    3    13     7    10 7.9253e-01  1.0e+00    5.9e-02 (f[    3])
    4    21     2     8 7.8799e-01  2.8e-02    7.9e-03 (f[    3])
pn_search terminated: no basis change.

Major Iteration Log
major minor  func  grad  residual    step  type prox    inorm  (label)
    0     0    22     5 7.8799e-01           I 7.1e-03 7.0e-01 (f[    3])
    1     3    23     6 3.8709e-04  1.0e+00 SM 2.9e-03 2.5e-04 (f[    4])
    2     1    24     7 6.3081e-09  1.0e+00 SO 3.9e-05 4.2e-09 (f[    5])

Major Iterations. . . . 2
Minor Iterations. . . . 4
Restarts. . . . . . . . 0
Crash Iterations. . . . 4
Gradient Steps. . . . . 0


In [6]:
solution_summary(compl; verbose = true)

* Solver : Path 5.0.03

* Status
  Result count       : 1
  Termination status : LOCALLY_SOLVED
  Message from the solver:
  "The problem was solved"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : NO_SOLUTION
  Objective value    : 0.00000e+00
  Primal solution :
    p_star[1] : 1.10682e-02
    p_star[2] : 1.16858e-02
    p_star[3] : 1.07203e-02
    pii_star[1] : 1.60734e-02
    pii_star[2] : 4.19819e-02
    pii_star[3] : 0.00000e+00
    pii_star[4] : 3.10666e-02
    z_star[1] : 1.00000e+00
    z_star[2] : 1.00000e+00
    z_star[3] : 1.00000e+00
    z_star[4] : 1.00000e+00
    z_star[5] : 1.00000e+00

* Work counters
  Solve time (sec)   : 1.56000e-01


In [7]:
# SU-NEO - LP
# Instantiate the model
primal = Model(GLPK.Optimizer)
(@isdefined primal) && (primal = nothing; primal = Model(GLPK.Optimizer));
# Define the optimisation model
@variable(primal, z_star[1:size(V,1)] >= 0, base_name = "z*")
@variable(primal, c_star >= 0, base_name = "c*")
@objective(primal, Max, c_star)
@constraint(primal, c1, (model_data.V'-model_data.U)*(z_star) .>= c_star*model_data.e)
@constraint(primal, c2, model_data.F*z_star .<= model_data.ϕ);
# Run the model and show results
optimize!(primal)
@show termination_status(primal)
@show primal_status(primal)
@show dual_status(primal)
@show objective_value(primal)
@show value.(z_star)
@show value.(c_star)
@show value.(c1)
@show value.(c2)
@show shadow_price.(c1)
@show shadow_price.(c2)

termination_status(primal) = MathOptInterface.OPTIMAL
primal_status(primal) = MathOptInterface.FEASIBLE_POINT
dual_status(primal) = MathOptInterface.FEASIBLE_POINT
objective_value(primal) = 0.9999999999999999
value.(z_star) = [0.9999999999999996, 0.9999999999999998, 1.0, 0.9999999999999992, 1.0000000000000002]
value.(c_star) = 0.9999999999999999
value.(c1) = [0.0; 0.0; 0.0;;]
value.(c2) = [27.499999999999996; 6.666666666666666; 8.148148148148147; 21.428571428571427;;]
shadow_price.(c1) = [0.007976425700459118; 0.008421527271114592; 0.0077257465770894235;;]
shadow_price.(c2) = [0.01158350920034474; 0.03025480355223982; 0.0; 0.022388557643305204;;]


4×1 Matrix{Float64}:
 0.01158350920034474
 0.03025480355223982
 0.0
 0.022388557643305204

In [8]:
#SU-NEO - LP
# Instantiate the model
dual = Model(GLPK.Optimizer)
(@isdefined dual) && (dual = nothing; dual = Model(GLPK.Optimizer));

# Set up the model
dual = Model(GLPK.Optimizer)

# Define the optimisation model
@variable(dual, p[1:size(sut.V,2)] >= 0)
@variable(dual, pii[1:size(sut.F,1)] >= 0)
@objective(dual, Min, sum(pii'*model_data.ϕ))
@constraint(dual, c1d, (model_data.V'-model_data.U)'*(p) .<= model_data.F'*pii)
@constraint(dual, c2d, (p'*model_data.e) .== sum(model_data.e))# or .==1?

# Run the model and show results
optimize!(dual)
model_solution(dual)
@show value.(p)
@show value.(pii)
@show shadow_price.(c1d)
@show shadow_price.(c2d);

termination_status(model) = MathOptInterface.OPTIMAL
primal_status(model) = MathOptInterface.FEASIBLE_POINT
dual_status(model) = MathOptInterface.FEASIBLE_POINT
objective_value(model) = 124.0000000000001
value.(p) = [0.9890767868569321, 1.044269381618209, 0.9579925755590895]
value.(pii) = [1.4363551408427555, 3.751595640477734, 0.0, 2.7761811477698406]
shadow_price.(c1d) = [-1.000000000000008, -0.9999999999999943, -0.9999999999999949, -1.0000000000000138, -1.0]
shadow_price.(c2d) = [-0.999999999999998;;]


In [10]:
#SU-RCOT_abs
dosso_primal = Model(GLPK.Optimizer)
(@isdefined dosso_primal) && (dosso_primal = nothing; dosso_primal = Model(GLPK.Optimizer));

@variable(dosso_primal, z_star[1:size(V,1)] >= 0, base_name = "z*")
@objective(dosso_primal, Min, sum(model_data.pii'*model_data.F*z_star))
@constraint(dosso_primal, mass_balance, (model_data.V'-model_data.U)*(z_star) .>= model_data.e)
@constraint(dosso_primal, zero_profit, (model_data.F)*(z_star) .<= model_data.ϕ)
optimize!(dosso_primal)
model_solution(dosso_primal)
show_primal_lhs(dosso_primal)

termination_status(model) = MathOptInterface.OPTIMAL
primal_status(model) = MathOptInterface.FEASIBLE_POINT
dual_status(model) = MathOptInterface.FEASIBLE_POINT
objective_value(model) = 123.99999999999999
value.(var_con) = [0.9999999999999993, 1.0, 1.0000000000000002, 0.9999999999999996, 0.9999999999999988]
value.(demand_con) = [51.0, 42.0, 31.0]
value.(factor_con) = [27.499999999999993, 6.666666666666666, 8.148148148148147, 21.428571428571427]


4-element Vector{Float64}:
 27.499999999999993
  6.666666666666666
  8.148148148148147
 21.428571428571427

In [11]:
#SU-DOSSO_abs
dosso_primal = Model(GLPK.Optimizer)
(@isdefined dosso_primal) && (dosso_primal = nothing; dosso_primal = Model(GLPK.Optimizer));

@variable(dosso_primal, z_star[1:size(V,1)] >= 0, base_name = "z*")
@objective(dosso_primal, Min, sum(model_data.pii'*model_data.F*z_star))
@constraint(dosso_primal, mass_balance, (model_data.V'-model_data.U)*(z_star) .>= model_data.e)
#@constraint(dosso_primal, zero_profit, (model_data.F)*(z_star) .<= model_data.ϕ)
optimize!(dosso_primal)
model_solution(dosso_primal)
show_primal_lhs(dosso_primal)

termination_status(model) = MathOptInterface.OPTIMAL
primal_status(model) = MathOptInterface.FEASIBLE_POINT
dual_status(model) = MathOptInterface.FEASIBLE_POINT
objective_value(model) = 124.0
value.(var_con) = [0.15818532500891247, 1.2251524264833586, 1.3903489126758504, 0.0, 0.0]
value.(demand_con) = [51.0, 42.0, 31.0]
value.(factor_con) = Float64[]


Float64[]

In [30]:
# SU-NST - MCP
# To set up our initial primal, we write:
compl = Model(PATHSolver.Optimizer)

act = size(model_data.V,1)
com = size(model_data.V,2)
fac = size(model_data.F,1)

# Define the optimisation model
@variable(compl, z_star[1:act] >= 0)
@variable(compl, p_star[1:com] >= 0)
@variable(compl, pii_star[1:fac] >= 0)

@constraint(compl, c1, vec(pii_star'*model_data.F - (p_star'*(model_data.V' - model_data.U))) ⟂ z_star)
@constraint(compl, c2, vec((model_data.V' - model_data.U)*(z_star) - model_data.e) ⟂ p_star)
@constraint(compl, c3, vec((model_data.ϕ - model_data.F*z_star)) ⟂ pii_star)

# Solve
optimize!(compl)

Path 5.0.03 (Fri Jun 26 10:05:33 2020)
Written by Todd Munson, Steven Dirkse, Youngdae Kim, and Michael Ferris


Crash Log
major  func  diff  size  residual    step       prox   (label)
    0     0             1.4596e+02             0.0e+00 (f[    6])
    1    11     6     3 1.4595e+02  3.2e-04    1.5e+00 (f[    6])
    2    12     7     6 5.9007e+00  1.0e+00    1.3e+00 (f[   12])
    3    13     7    10 7.9253e-01  1.0e+00    5.9e-02 (f[    3])
    4    21     2     8 7.8799e-01  2.8e-02    7.9e-03 (f[    3])
pn_search terminated: no basis change.

Major Iteration Log
major minor  func  grad  residual    step  type prox    inorm  (label)
    0     0    22     5 7.8799e-01           I 7.1e-03 7.0e-01 (f[    3])
    1     3    23     6 3.8709e-04  1.0e+00 SM 2.9e-03 2.5e-04 (f[    4])
    2     1    24     7 6.3081e-09  1.0e+00 SO 3.9e-05 4.2e-09 (f[    5])

Major Iterations. . . . 2
Minor Iterations. . . . 4
Restarts. . . . . . . . 0
Crash Iterations. . . . 4
Gradient Steps. . . . . 0


In [31]:
solution_summary(compl; verbose = true)

* Solver : Path 5.0.03

* Status
  Result count       : 1
  Termination status : LOCALLY_SOLVED
  Message from the solver:
  "The problem was solved"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : NO_SOLUTION
  Objective value    : 0.00000e+00
  Primal solution :
    p_star[1] : 1.10682e-02
    p_star[2] : 1.16858e-02
    p_star[3] : 1.07203e-02
    pii_star[1] : 1.60734e-02
    pii_star[2] : 4.19819e-02
    pii_star[3] : 0.00000e+00
    pii_star[4] : 3.10666e-02
    z_star[1] : 1.00000e+00
    z_star[2] : 1.00000e+00
    z_star[3] : 1.00000e+00
    z_star[4] : 1.00000e+00
    z_star[5] : 1.00000e+00

* Work counters
  Solve time (sec)   : 3.10000e-02


In [32]:
sum(value.(pii_star)'*model_data.F)

1.387610577406891

In [33]:
value.(p_star)'*model_data.e

1×1 adjoint(::Vector{Float64}) with eltype Float64:
 1.3876105719834

In [37]:
value.(p_star)'*model_data.V'

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 1.20406  1.15357  0.952972  0.644085  0.462713

In [41]:
sum(value.(p_star)'*model_data.V')

4.417403009707454

In [40]:
value.(p_star)'*(sum(model_data.U,dims=2) + model_data.e)

1×1 adjoint(::Vector{Float64}) with eltype Float64:
 4.417403009707454

In [42]:
value.(p_star)'*(model_data.U) + value.(pii_star)'*model_data.F

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 1.20406  1.15357  0.952972  0.644085  0.462713

In [43]:
sum(value.(p_star)'*(model_data.U) + value.(pii_star)'*model_data.F)

4.417403015130944

In [ ]:
itc = Constructs.ITC(sut)

construct([136.0; 96.0; 167.0;;], [51.0; 42.0; 31.0;;], [9.641472868217054 12.97168343454485 62.38684369723809; 31.21884735202492 15.000403984873474 7.780748663101605; 55.291530826631885 47.03340135357168 33.675067819796425], [0.07089318285453716 0.1351217024431755 0.37357391435471915; 0.22955034817665385 0.15625420817576535 0.04659130935988985; 0.4065553737252344 0.48993126409970494 0.20164711269339178], [1.59833317316061 0.7144586737028918 0.7896046410737875; 0.49661851260095824 1.4487520746555465 0.3169312490909092; 1.11870313292003 1.2528995188783316 1.8491729170383386], [11.41180057475428 4.148556760568855 11.939642664676864; 1.8050172265288542 2.3776251212691895 2.484024318868623; 2.3464303251306076 0.5423046132692892 5.259413209748251; 6.343125890506894 2.8536048675862875 12.231840670478245], [0.08391029834378147 0.043214132922592234 0.0714948662555501; 0.013272185489182752 0.024766928346554046 0.014874397118973792; 0.017253164155372116 0.005649006388221764 0.031493492273941624;

In [ ]:
model_data = Model_data.IO(itc)

[ Info: You are setting up an IO model dataset. Elements of this dataset are now treated independently, meaning that no recalculation whatsoever takes place when individual elements are changed.


construct([136.0; 96.0; 167.0;;], [51.0; 42.0; 31.0;;], [9.641472868217054 12.97168343454485 62.38684369723809; 31.21884735202492 15.000403984873474 7.780748663101605; 55.291530826631885 47.03340135357168 33.675067819796425], [0.07089318285453716 0.1351217024431755 0.37357391435471915; 0.22955034817665385 0.15625420817576535 0.04659130935988985; 0.4065553737252344 0.48993126409970494 0.20164711269339178], [1.59833317316061 0.7144586737028918 0.7896046410737875; 0.49661851260095824 1.4487520746555465 0.3169312490909092; 1.11870313292003 1.2528995188783316 1.8491729170383386], [11.41180057475428 4.148556760568855 11.939642664676864; 1.8050172265288542 2.3776251212691895 2.484024318868623; 2.3464303251306076 0.5423046132692892 5.259413209748251; 6.343125890506894 2.8536048675862875 12.231840670478245], [0.08391029834378147 0.043214132922592234 0.0714948662555501; 0.013272185489182752 0.024766928346554046 0.014874397118973792; 0.017253164155372116 0.005649006388221764 0.031493492273941624;

In [12]:
# IO-NST - MCP
# To set up our initial primal, we write:
compl = Model(PATHSolver.Optimizer)

act = size(model_data.A,1)
com = size(model_data.A,2)
fac = size(model_data.R,1)

# Define the optimisation model
@variable(compl, x_star[1:act] >= 0)
@variable(compl, p_star[1:com] >= 0)
@variable(compl, pii_star[1:fac] >= 0)

@constraint(compl, c1, vec(pii_star'*model_data.R - (p_star'*(model_data.I_mod - model_data.A))) ⟂ x_star)
@constraint(compl, c2, vec((model_data.I_mod - model_data.A)*(x_star) - model_data.y) ⟂ p_star)
@constraint(compl, c3, vec((model_data.ϕ - model_data.R*x_star)) ⟂ pii_star)

# Solve
optimize!(compl)

Path 5.0.03 (Fri Jun 26 10:05:33 2020)
Written by Todd Munson, Steven Dirkse, Youngdae Kim, and Michael Ferris


Crash Log
major  func  diff  size  residual    step       prox   (label)
    0     0             1.4596e+02             0.0e+00 (f[    4])
    1     1     6     3 1.1819e+02  1.0e+00    1.5e+00 (f[    4])
    2     2     3     6 9.5511e+01  1.0e+00    1.2e+00 (f[    6])
    3     3     0     6 8.2900e+01  1.0e+00    9.6e-01 (f[    3])
pn_search terminated: no basis change.

Major Iteration Log
major minor  func  grad  residual    step  type prox    inorm  (label)
    0     0     4     4 8.2900e+01           I 8.3e-01 4.1e+01 (f[    3])
    1     1     5     5 5.5202e+01  1.0e+00 SO 3.3e-01 4.2e+01 (f[    3])
    2     5     6     6 1.6822e+01  1.0e+00 SO 1.3e-01 7.4e+00 (f[    6])
    3     1     7     7 2.7560e+00  1.0e+00 SO 5.3e-02 1.7e+00 (f[    3])
    4     1     8     8 1.7326e-01  1.0e+00 SO 2.1e-02 7.9e-02 (f[    3])
    5     1     9     9 4.3167e-03  1.0e+00 SO 8.

In [13]:
solution_summary(compl; verbose = true)

* Solver : Path 5.0.03

* Status
  Result count       : 1
  Termination status : LOCALLY_SOLVED
  Message from the solver:
  "The problem was solved"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : NO_SOLUTION
  Objective value    : 0.00000e+00
  Primal solution :
    p_star[1] : 1.99387e+01
    p_star[2] : 1.87402e+01
    p_star[3] : 1.92579e+01
    pii_star[1] : 4.94781e+01
    pii_star[2] : 1.19960e+01
    pii_star[3] : 1.49669e+01
    pii_star[4] : 3.91247e+01
    x_star[1] : 1.36000e+02
    x_star[2] : 9.60000e+01
    x_star[3] : 1.67000e+02

* Work counters
  Solve time (sec)   : 3.20000e-02


In [21]:
sum(value.(pii_star)'*model_data.G)

2400.9594420674325

In [19]:
value.(p_star)'*model_data.y

1×1 adjoint(::Vector{Float64}) with eltype Float64:
 2400.9594420658314

In [23]:
value.(p_star)'*(sum(model_data.Z,dims=2) + model_data.y)

1×1 adjoint(::Vector{Float64}) with eltype Float64:
 7726.798622507047

In [24]:
value.(p_star)'*model_data.x

1×1 adjoint(::Vector{Float64}) with eltype Float64:
 7726.798622507047

In [26]:
sum(value.(p_star)'*(model_data.Z) + value.(pii_star)'*model_data.G)

7726.798622508648

In [27]:
value.(p_star)'*value.(x_star)

7726.79862250758

# Increase \varphi

In [59]:
sum(model_data.V'-model_data.U,dims=2)-model_data.e

3×1 Matrix{Float64}:
 0.0
 0.0
 0.0

In [53]:
model_data = Model_data.SU(sut);

[ Info: You are setting up an SU model dataset. Elements of this dataset are now treated independently, meaning that no recalculation whatsoever takes place when individual elements are changed.


In [54]:
model_data.ϕ

4×1 Matrix{Float64}:
 27.499999999999996
  6.666666666666666
  8.148148148148147
 21.428571428571427

In [50]:
model_data.ϕ[2:4] = model_data.ϕ[2:4].+2

3-element Vector{Float64}:
  8.666666666666666
 10.148148148148147
 23.428571428571427

In [61]:
# SU-NST - MCP
# To set up our initial primal, we write:
compl = Model(PATHSolver.Optimizer)

act = size(model_data.V,1)
com = size(model_data.V,2)
fac = size(model_data.F,1)

# Define the optimisation model
@variable(compl, z_star[1:act] >= 0)
@variable(compl, p_star[1:com] >= 0)
@variable(compl, pii_star[1:fac] >= 0)

@constraint(compl, c1, vec(pii_star'*model_data.F - (p_star'*(model_data.V' - model_data.U))) ⟂ z_star)
@constraint(compl, c2, vec((model_data.V' - model_data.U)*(z_star) - model_data.e) ⟂ p_star)
@constraint(compl, c3, vec((model_data.ϕ - model_data.F*z_star)) ⟂ pii_star)

# Solve
optimize!(compl)

Path 5.0.03 (Fri Jun 26 10:05:33 2020)
Written by Todd Munson, Steven Dirkse, Youngdae Kim, and Michael Ferris


Crash Log
major  func  diff  size  residual    step       prox   (label)
    0     0             1.4596e+02             0.0e+00 (f[    6])
    1    11     6     3 1.4595e+02  3.2e-04    1.5e+00 (f[    6])
    2    12     7     6 5.9007e+00  1.0e+00    1.3e+00 (f[   12])
    3    13     7    10 7.9253e-01  1.0e+00    5.9e-02 (f[    3])
    4    21     2     8 7.8799e-01  2.8e-02    7.9e-03 (f[    3])
pn_search terminated: no basis change.

Major Iteration Log
major minor  func  grad  residual    step  type prox    inorm  (label)
    0     0    22     5 7.8799e-01           I 7.1e-03 7.0e-01 (f[    3])
    1     3    23     6 3.8709e-04  1.0e+00 SM 2.9e-03 2.5e-04 (f[    4])
    2     1    24     7 6.3081e-09  1.0e+00 SO 3.9e-05 4.2e-09 (f[    5])

Major Iterations. . . . 2
Minor Iterations. . . . 4
Restarts. . . . . . . . 0
Crash Iterations. . . . 4
Gradient Steps. . . . . 0


In [62]:
solution_summary(compl; verbose = true)

* Solver : Path 5.0.03

* Status
  Result count       : 1
  Termination status : LOCALLY_SOLVED
  Message from the solver:
  "The problem was solved"

* Candidate solution (result #1)
  Primal status      : FEASIBLE_POINT
  Dual status        : NO_SOLUTION
  Objective value    : 0.00000e+00
  Primal solution :
    p_star[1] : 1.10682e-02
    p_star[2] : 1.16858e-02
    p_star[3] : 1.07203e-02
    pii_star[1] : 1.60734e-02
    pii_star[2] : 4.19819e-02
    pii_star[3] : 0.00000e+00
    pii_star[4] : 3.10666e-02
    z_star[1] : 1.00000e+00
    z_star[2] : 1.00000e+00
    z_star[3] : 1.00000e+00
    z_star[4] : 1.00000e+00
    z_star[5] : 1.00000e+00

* Work counters
  Solve time (sec)   : 0.00000e+00


In [ ]:
sum(value.(pii_star)'*model_data.F)

1.387610577406891

In [ ]:
value.(p_star)'*model_data.e

1×1 adjoint(::Vector{Float64}) with eltype Float64:
 1.3876105719834

In [ ]:
value.(p_star)'*model_data.V'

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 1.20406  1.15357  0.952972  0.644085  0.462713

In [ ]:
sum(value.(p_star)'*model_data.V')

4.417403009707454

In [ ]:
value.(p_star)'*(sum(model_data.U,dims=2) + model_data.e)

1×1 adjoint(::Vector{Float64}) with eltype Float64:
 4.417403009707454

In [ ]:
value.(p_star)'*(model_data.U) + value.(pii_star)'*model_data.F

1×5 adjoint(::Vector{Float64}) with eltype Float64:
 1.20406  1.15357  0.952972  0.644085  0.462713

In [ ]:
sum(value.(p_star)'*(model_data.U) + value.(pii_star)'*model_data.F)

4.417403015130944